DDPはDPとは違い，ラッピングするだけとはいかない．

重要な箇所のみ解説する

また，このコードをそのままcolab上で動かすことはおそらくできないのでaeonなどのマシンを使って実装するように．

In [ ]:
def main():
    ngpus_per_node = torch.cuda.device_count()
    mp.spawn(main_worker, nprocs=ngpus_per_node, args=(ngpus_per_node, args))

mp.spawn(A,B,C)を使ってプロセスを複数個発生させる

これは，実際には関数A(X,B,C)を複数行っている

ここでのXは各GPU情報になる(たとえばGPU:0のような形)

In [ ]:
def main_worker(gpu, ngpus_per_node, args):
    args.gpu = gpu
    dist.init_process_group(
    backend="nccl",
    init_method="tcp://172.17.0.3:8888",
    world_size=ngpus_per_node,
    rank=args.gpu
    )
    torch.cuda.set_device(args.gpu)
    model = eval(args.model)(100)
    model.cuda(args.gpu)
    # When using a single GPU per process and per
    # DistributedDataParallel, we need to divide the batch size
    # ourselves based on the total number of GPUs of the current node.
    args.batch_size = int(args.batch_size / ngpus_per_node)
    args.workers = int((args.num_worker + ngpus_per_node - 1) / ngpus_per_node)
    model = torch.nn.parallel.DistributedDataParallel(
        model, device_ids=[args.gpu]
    )
    averaged_model = AveragedModel(model)

    loss_func = nn.CrossEntropyLoss().cuda(args.gpu)

    optimizer = torch.optim.SGD(
        model.parameters(),
        args.lr,
        momentum=args.momentum,
        weight_decay=args.weight_decay,
    )
    ................................................(省略)

大事になるのがdist.init_process_groupの部分でここで各GPUの通信が初期化される．

backendはGPUを利用する場合はncclで固定

init_methodはマシンのIPアドレスを入力(localhostとして127.0.0.1でもおk)

また，modelをtorch.nn.parallel.DistributedDataParalleでラッピングする必要がある．

これ以降は通常の学習と同様の手順で進めていく


最後にDDP用のコードを共有して終わる．

このコードは先ほどと同様にcolab上では動かないので注意すること．


In [ ]:
# -*- coding: utf-8 -*-
import argparse
import sys
import time

import torch
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.multiprocessing as mp
import torch.optim as optim
import torch.nn as nn

from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim.lr_scheduler import MultiStepLR
from torch.optim.swa_utils import AveragedModel

from model.wide_res_net import WRN28_10
from model.pyramid import Pyramid
from model.resnet import ResNet18,ResNet50,ResNet101,ResNet152
from model.smooth_cross_entropy import smooth_crossentropy

from data.cifar import CIFAR10,CIFAR100

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

from utility.cutout import Cutout

parser = argparse.ArgumentParser(description='PyTorch Training')
parser.add_argument('--model',default='ResNet18',type=str)
parser.add_argument('--dataset',default='CIFAR100',type=str)
#parser.add_argument('--optimizer',default='SGD',type=str,help='SGD')
parser.add_argument('--lr_type', default='MultiStepLR', type=str)#'MultiStepLR'/'CosineAnnealingLR'
parser.add_argument('--epoch', default=10, type=int)
parser.add_argument('--start_averaged', default=2, type=int)
parser.add_argument('--batch_size', default=256, type=int)
parser.add_argument('--weight_decay', default=5e-3, type=float)
parser.add_argument('--momentum', default=0.0, type=float)
parser.add_argument('--lr', default=0.1, type=float)
parser.add_argument('--eta_min',default=0.00,type=float)
parser.add_argument('--num_worker', default=8, type=int, help='number of workers')
parser.add_argument('--label_smoothing',default=0.0,type=float)
parser.add_argument('--loss_func', default='CE', type=str)
parser.add_argument('--debug_flag',default='OFF',type=str)
parser.add_argument('--start_cm', default=300, type=int)
#'SCE/CE'



args = parser.parse_args()

total_train=50000
total_test=10000

def main():
    ngpus_per_node = torch.cuda.device_count()
    mp.spawn(main_worker, nprocs=ngpus_per_node, args=(ngpus_per_node, args))

def main_worker(gpu, ngpus_per_node, args):
    args.gpu = gpu
    dist.init_process_group(
    backend="nccl",
    init_method="tcp://172.17.0.3:8888",
    world_size=ngpus_per_node,
    rank=args.gpu
    )
    torch.cuda.set_device(args.gpu)
    model = eval(args.model)(100)
    model.cuda(args.gpu)
    # When using a single GPU per process and per
    # DistributedDataParallel, we need to divide the batch size
    # ourselves based on the total number of GPUs of the current node.
    args.batch_size = int(args.batch_size / ngpus_per_node)
    args.workers = int((args.num_worker + ngpus_per_node - 1) / ngpus_per_node)
    model = torch.nn.parallel.DistributedDataParallel(
        model, device_ids=[args.gpu]
    )
    averaged_model = AveragedModel(model)

    loss_func = nn.CrossEntropyLoss().cuda(args.gpu)

    optimizer = torch.optim.SGD(
        model.parameters(),
        args.lr,
        momentum=args.momentum,
        weight_decay=args.weight_decay,
    )
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=args.epoch)

    train_set = torchvision.datasets.CIFAR100(root='./cifar', train=True, download=True, transform=transforms.ToTensor())
    data = torch.cat([d[0] for d in DataLoader(train_set)])
    mean,std = data.mean(dim=[0, 2, 3]), data.std(dim=[0, 2, 3])

    train_transform = transforms.Compose([
        torchvision.transforms.RandomCrop(size=(32, 32), padding=4),
        torchvision.transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
        Cutout()
    ])

    test_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])

    train_dataset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=train_transform)
    test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=test_transform)
    train_sampler = torch.utils.data.distributed.DistributedSampler(train_dataset)
    test_sampler = torch.utils.data.distributed.DistributedSampler(
        test_dataset, shuffle=False, drop_last=True
    )
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=args.batch_size, shuffle=(train_sampler is None), num_workers=args.workers,pin_memory=True,sampler=train_sampler)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False, num_workers=args.workers,pin_memory=True,sampler=test_sampler)
    total_time = 0
    for epoch in range(args.epoch):
        torch.cuda.synchronize()
        time_ep = time.time()
        train_loss, train_acc, time_ep = train(epoch,model,averaged_model,loss_func,train_loader, optimizer,args)
        scheduler.step()
        print("epoch", epoch+1, "lr:{:.7f}".format(optimizer.param_groups[0]['lr']), " train_loss:{:.5f}".format(train_loss), "train_acc:{:.2f}".format(train_acc), "time:{:.3f}".format(time_ep))
        total_time += time_ep

        if (epoch + 1)%5 == 0:
            test_loss, test_acc, avg_test_loss, avg_test_acc= test(epoch,model,averaged_model,loss_func,test_loader,args)
            print('test_loss:{:.5f}'.format(test_loss), 'test_acc:{:.2f}'.format(test_acc), 'avg_test_loss:{:.5f}'.format(avg_test_loss), 'avg_test_acc:{:.2f}'.format(avg_test_acc))
    print('{:.0f}:{:.0f}:{:.0f}'.format(total_time//3600, total_time%3600//60, total_time%3600%60))
    if args.gpu == 0:
        torch.save(model.state_dict(),'/home/noise_sam/DDP/model.pth')
        torch.save(averaged_model.state_dict(),'/home/noise_sam/DDP/averaged_model.pth')

def train(epoch,model,averaged_model,loss_func,train_loader, optimizer,args):

    torch.cuda.synchronize()
    time_ep = time.time()
    model.train()
    averaged_model.train()

    train_loss = 0.0
    correct = 0.0

    for inputs, target in train_loader:
        optimizer.zero_grad()
        input_size = inputs.size()[0]
        inputs, target = inputs.cuda(), target.cuda()
        output = model(inputs)
        loss = loss_func(output, target)

        if args.loss_func == "SCE":
            loss.mean().backward()
            optimizer.step()
            if epoch >= args.start_averaged:
                averaged_model.update_parameters(model)
            with torch.no_grad():
                train_loss += loss.sum().item()/total_train
                correct += (torch.argmax(output, 1) == target).sum().item()/total_train
        else:
            loss.backward()
            optimizer.step()
            if epoch >= args.start_averaged:
                averaged_model.update_parameters(model)
            with torch.no_grad():
                train_loss += loss.data * input_size/total_train
                _, pred = torch.max(output, 1)
                correct += pred.eq(target).sum()/total_train
    torch.cuda.synchronize()
    time_ep = time.time() - time_ep

    if epoch >= args.start_averaged:
        torch.optim.swa_utils.update_bn(train_loader, averaged_model, optimizer.param_groups[0]['params'][0].device)


    return train_loss, 100*correct, time_ep

def test(epoch,model,averaged_model,loss_func,test_loader,args):
    model.eval()
    averaged_model.eval()

    test_loss = 0.0
    test_correct = 0.0
    avg_test_loss = 0.0
    avg_test_correct = 0.0

    with torch.no_grad():
        for inputs, target in test_loader:
            input_size = inputs.size()[0]
            inputs, target = inputs.cuda(), target.cuda()

            if args.loss_func == "SCE":
                #normal_prediction
                predict = model(inputs)
                loss = loss_func(predict, target)
                test_loss += loss_func(predict, target).sum().item()/ total_test
                test_correct += (torch.argmax(predict, 1) == target).sum().item() / total_test
                #averaged_prediction
                avg_predict = averaged_model(inputs)
                loss = loss_func(avg_predict, target)
                avg_test_loss += loss_func(avg_predict, target).sum().item()/ total_test
                avg_test_correct += (torch.argmax(avg_predict, 1) == target).sum().item() / total_test
            else:
                #normal_prediction
                predict = model(inputs)
                loss = loss_func(predict, target)
                test_loss += loss.data * input_size / total_test
                _, pred = torch.max(predict.data, 1)
                test_correct += pred.eq(target).sum() / total_test
                #averaged_prediction
                avg_predict = averaged_model(inputs)
                loss = loss_func(avg_predict, target)
                avg_test_loss += loss.data * input_size / total_test
                _, pred = torch.max(avg_predict.data, 1)
                avg_test_correct += pred.eq(target).sum() / total_test

    return test_loss, 100*test_correct, avg_test_loss, 100*avg_test_correct

def copy_model(epoch):
    for p,avg_p in zip(model.parameters(),averaged_model.parameters()):
        avg_p_ = avg_p.detach()
        p.detach().copy_(avg_p_)
    torch.optim.swa_utils.update_bn(dataset.train, model, optimizer.param_groups[0]['params'][0].device)
    print('======================================')
    test_loss, test_acc, avg_test_loss, avg_test_acc= test(epoch)
    print('test_loss:{:.5f}'.format(test_loss), 'test_acc:{:.2f}'.format(test_acc), 'avg_test_loss:{:.5f}'.format(avg_test_loss), 'avg_test_acc:{:.2f}'.format(avg_test_acc))

def norm_avg():
    for named_p,avg_p in zip(model.named_parameters(),averaged_model.parameters()):
        name,p = named_p
        if not 'norm' in name:
            p_ = p.detach()
            avg_p.detach().copy_(p_)
    torch.optim.swa_utils.update_bn(dataset.train, averaged_model, optimizer.param_groups[0]['params'][0].device)
    print('======================================')
    test_loss, test_acc, avg_test_loss, avg_test_acc= test(epoch)
    print('test_loss:{:.5f}'.format(test_loss), 'test_acc:{:.2f}'.format(test_acc), 'avg_test_loss:{:.5f}'.format(avg_test_loss), 'avg_test_acc:{:.2f}'.format(avg_test_acc))

if __name__ == "__main__":
    main()